<a href="https://colab.research.google.com/github/aliash98/Search2Vec-BScThesis/blob/main/Implementation%20on%20AOL%20Dataset/AOL_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datetime import datetime
import pandas as pd
import numpy as np
import re
import json
import warnings
import gc 
from tqdm import tqdm, trange
from scipy import spatial
import math

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
df = pd.read_csv(r'./gdrive/MyDrive/Colab Notebooks/AOL_Implementation/weights.tsv', sep="\t")   
weights = df.values

# *** Note : this weights matrix doesn't contain the 0th redundant index

print(weights.shape)
print(weights[:5, :5])

(7895, 300)
[[-0.02851121 -0.03439867 -0.03944187  0.01007509 -0.02832736]
 [-0.02665158 -0.00733381 -0.07140837 -0.00545206  0.03360269]
 [-0.03598765 -0.05363363  0.02089478  0.06033646 -0.04562914]
 [-0.02651295  0.00572839 -0.04466211  0.05167326 -0.05439606]
 [-0.08069196  0.07970319  0.06144302 -0.07563543 -0.10983358]]


In [ ]:
vocab = json.load( open( r'./gdrive/MyDrive/Colab Notebooks/AOL_Implementation/vocab.json' ) )
print(type(vocab))
inverse_vocab = {index: token for token, index in vocab.items()}
print(inverse_vocab[1])

<class 'dict'>
emma watson


In [ ]:
# Shifting the vocab
vocab_cor = {}
for i in range(1, len(inverse_vocab)):
  vocab_cor[inverse_vocab[i]] = i - 1

inverse_vocab_cor = {index: token for token, index in vocab_cor.items()}
print(inverse_vocab[0])

<pad>


In [ ]:
vocab_cor['emma watson']

0

In [ ]:
inverse_vocab_cor[4979]

'www.asiantown.net'

In [ ]:
# Please specify the correct number of queries and items right here

# 4981 unique queries + 2915 unique urls

vocab_size, dim = weights.shape
number_of_uurls = 2915
number_of_uqueries = 4980
distances = np.zeros((number_of_uqueries, number_of_uurls), dtype=float)
def all_cosine_distances():
  for i in range(0, number_of_uqueries):
    for j in range(number_of_uurls):
      distances[i, j] = spatial.distance.cosine(weights[j + number_of_uqueries, :], weights[i, :])
    if i % 100 == 0:
      print(i)
all_cosine_distances()
print(weights.shape)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
(7895, 300)


In [ ]:
nearest_cat_vocab = {}
counter = 0
x, y = distances.shape
print(x)
print(y)
for i in range(x):
  temp_list = []
  temp_arr = distances[i, :]
  for j in range(10):
    temp_index = np.argmin(temp_arr)
    if i == 122:
      print(temp_index + x)
      print(temp_arr[temp_index])
    temp_list.append(inverse_vocab_cor[x + temp_index])
    temp_arr[temp_index] = 1

  nearest_cat_vocab[inverse_vocab_cor[i]] = temp_list


4980
2915
5349
0.2654277835286887
5129
0.2802815371384506
5350
0.32548910499083517
5137
0.3323852881512671
5317
0.3855190411820817
6671
0.40206260678402284
7576
0.41616606171424353
5136
0.46498239980723455
6665
0.5502567098196287
4983
0.641463963165277


In [ ]:
print(inverse_vocab_cor[12495])
print(inverse_vocab_cor[122])
print(nearest_cat_vocab[inverse_vocab_cor[122]])

In [ ]:
# Not necessary

def least_cosine_distance(index, k):
  vocab_size, dim = weights.shape
  vector = weights[index, :]
  # print(vector.shape)
  # lets find the k nearest point to this
  distances = []
  indexes = []
  min_distances = []
  for i in range(0, vocab_size):
    distances.append(spatial.distance.cosine(vector, weights[i, :]))
  counter = 0
  while counter < k:
    temp_index = np.argmin(distances)
    if temp_index == index:
      distances[temp_index] = 1
      continue
    if not inverse_vocab[temp_index].endswith('-url'):
      distances[temp_index] = 1
      continue
    indexes.append(temp_index + 1)
    min_distances.append(distances[temp_index])
    distances[temp_index] = 1
    counter = counter + 1
  return indexes, min_distances 

In [ ]:
my_point = 123
indexes, min_distances = least_cosine_distance(my_point - 1, 10)
print(inverse_vocab[my_point])
print(min_distances)
print(indexes)
for index in indexes:
  print(inverse_vocab[index])
# a = weights[33, :]
# b = weights[714, :]
# print(1 - np.dot(a, b)/(np.linalg.norm(a) * np.linalg.norm(b)))

anna benson
[0.09910247357259117, 0.14348363992977942, 0.46863742095863814, 0.7979733532030139, 0.8019996059999295, 0.8121298074720742, 0.8235685943652963, 0.8273066896916158, 0.828901816384344, 0.8298963963907586]
[12652, 12574, 13092, 13526, 14141, 13749, 14487, 14652, 13950, 14683]
http://www.annabenson.net-url
http://www.gorillamask.net-url
http://www.askmen.com-url
http://www.pcplanets.com-url
http://www.fhmus.com-url
http://www.annualcreditreport.com-url
http://redsox.mlb.com-url
http://www.anothersite.co.uk-url
http://www.easygals.com-url
http://www.wpunj.edu-url


In [ ]:
test = pd.read_csv (r'./gdrive/MyDrive/Colab Notebooks/AOL_Implementation/test.csv')

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137832 entries, 0 to 137831
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   AnonID     137832 non-null  int64  
 1   Query      137832 non-null  object 
 2   QueryTime  137832 non-null  object 
 3   ItemRank   62043 non-null   float64
 4   ClickURL   62043 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 5.3+ MB


In [ ]:
test.dropna(subset = ["ClickURL"], inplace=True)
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62043 entries, 1 to 137826
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   AnonID     62043 non-null  int64  
 1   Query      62043 non-null  object 
 2   QueryTime  62043 non-null  object 
 3   ItemRank   62043 non-null  float64
 4   ClickURL   62043 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 2.8+ MB


In [ ]:
test_optimized = test.iloc[:1].copy()
test_optimized.loc[0] = test.iloc[3].copy()
# test_optimized

counter = 0
temp_query = ''
temp_id = -1
for index, row in test.iterrows():
  if row['Query'] == '-':
    continue
  if row['AnonID'] == temp_id and row['Query'] == temp_query:
    continue
  temp_query = row['Query']
  temp_id = row['AnonID']
  test_optimized.loc[counter] = row
  counter = counter + 1
  if index % 1000 == 0:
    print(index)
    # if index > 1000:
    #   break

test_optimized.head(10)

4000
10000
23000
24000
26000
35000
36000
44000
50000
79000
85000
103000
107000
108000
130000
135000
136000


,AnonID,Query,QueryTime,ItemRank,ClickURL
1,507,pogo,2006-05-25 23:21:59,1.0,http://www.pogo.com
0,507,craigslist,2006-05-25 19:28:30,11.0,http://atlanta.craigslist.org
2,1020,encyclopedia,2006-05-30 12:41:04,1.0,http://www.encyclopedia.com
3,2708,wetcircle.com,2006-05-22 06:47:42,1.0,http://www.wetcircle.com
4,2729,google.com,2006-05-22 08:42:22,1.0,http://www.google.com
5,6161,paragonfcu,2006-05-27 10:38:47,1.0,http://www.paragonfcu.org
6,7597,google. com,2006-05-30 19:25:13,1.0,http://www.google.com
7,10133,skylight bank,2006-05-25 09:59:37,1.0,http://www.skylight.net
8,10133,yahoo messenger,2006-05-29 21:49:54,3.0,http://www.yahoo.com
9,10189,annualcreditreport.com,2006-05-31 00:34:01,6.0,http://www.freecreditreport.com


In [ ]:
test_optimized.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18020 entries, 1 to 18019
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   AnonID     18020 non-null  int64  
 1   Query      18020 non-null  object 
 2   QueryTime  18020 non-null  object 
 3   ItemRank   18020 non-null  float64
 4   ClickURL   18020 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 844.7+ KB


In [ ]:
test_optimized.to_csv(r'./gdrive/MyDrive/Colab Notebooks/AOL_Implementation/test_optimized.csv', index=False)

In [ ]:
test_optimized = pd.read_csv (r'./gdrive/MyDrive/Colab Notebooks/AOL_Implementation/test_optimized.csv')

In [ ]:
# Not necessary

nearest_cat_vocab = {}
counter = 0
for word in vocab.keys():
  if not word.endswith('-url'):
    counter += 1
    if counter % 100 == 0:
      print(counter)
    # indexes, min_distances = least_cosine_distance(int(vocab[word]) - 1, 10)

    # finding k minimum distances

    temp_list = []
    for index in indexes:
      temp_list.append(inverse_vocab[index])
    nearest_cat_vocab[word] = temp_list
  
print(nearest_cat_vocab)

In [ ]:
mrr = 0
mrr_default_method = 0
not_found = 0
first_hit = 0
first_hit_default_method = 0
# zero_counter = 0
url_not_in_vocab = 0

for index, row in test_optimized.iterrows():
  temp_query = row['Query']
  temp_url = row['ClickURL'] + '-url'
  default_rank = int(row['ItemRank'])
  # if default_rank == 0:
  #   zero_counter += 1
  if default_rank < 11 and default_rank > 0:
    mrr_default_method += 1 / default_rank
    if default_rank == 1:
      first_hit_default_method += 1
  if temp_query in nearest_cat_vocab.keys():
    if temp_url in vocab_cor.keys(): 
      for j in range(10):  
        if nearest_cat_vocab[temp_query][j] == temp_url:
          if j == 0:
            first_hit = first_hit + 1
          mrr = mrr + 1 / (j+1)
          break
    else:
      url_not_in_vocab += 1
  else:
    not_found = not_found + 1

# print(zero_counter)
print("Number of Rows: " + str(test_optimized.shape[0]))
print("URL Not in Vocab: " + str(url_not_in_vocab))
print("Query Not in Vocab: " + str(not_found))
print("Number of vocab-existing queries: " + str(test_optimized.shape[0] - not_found))
print("MRR: " + str(mrr / test_optimized.shape[0]))
print("MRR in vocab-existing queries: " + str(mrr / (test_optimized.shape[0] - not_found) ))
print("First Hit: " + str(first_hit))
print("Default MRR: " + str(mrr_default_method / test_optimized.shape[0]))
print("Default First Hit: " + str(first_hit_default_method))

Number of Rows: 18020
URL Not in Vocab: 1638
Query Not in Vocab: 494
Number of vocab-existing queries: 17526
MRR: 0.7829900859714988
MRR in vocab-existing queries: 0.8050599879725213
First Hit: 13165
Default MRR: 0.8536299834399292
Default First Hit: 14061
